In [ ]:
# one dimensional problem with invertible net with two dimensions 
# referee that scales slopes down
# connection: slopes -> distance
# again: distance function is implemented by network
# 
# 

In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png' 
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 7
import seaborn
seaborn.set_style('darkgrid')

In [ ]:
from numpy.random import RandomState
from braindecode.torch_ext.util import np_to_var, var_to_np
x = np_to_var(np.array([0,1])[:,None], dtype=np.float32)
x_val = np_to_var(np.array([0.25,0.75])[:,None], dtype=np.float32)

In [ ]:
import torch.nn as nn
from torch.nn import ConstantPad2d
from reversible.revnet import ReversibleBlockOld, SubsampleSplitter, ViewAs


In [ ]:
def rev_block(n_chans, n_intermediate_chans, kernel_size=3):
    c = n_chans // 2
    n_i_c = n_intermediate_chans
    assert kernel_size % 2 == 1
    padding = kernel_size // 2
    return ReversibleBlockOld(
        nn.Sequential(
            nn.Linear(c, n_i_c),
             nn.ReLU(),
             nn.Linear(n_i_c, c)),
        nn.Sequential(
            nn.Linear(c, n_i_c),
             nn.ReLU(),
             nn.Linear(n_i_c, c)))


In [ ]:
feature_model = nn.Sequential(
        rev_block(2, 100),
        rev_block(2, 100),
)

In [ ]:
import torch as th
xx = th.cat((x,x), dim=1)
xx_val = th.cat((x_val,x_val), dim=1)

In [ ]:

outs = feature_model(xx)

In [ ]:
# show if inverting what happens
# which loss? first discriminator loss, but now a discriminator without spectral norm

from reversible.revnet import invert

In [ ]:
invert(feature_model,outs)

In [ ]:
# generate uniform noise between 0 and 2
# tell discriminator to distingiush by putting high values on real data, low values on fake data


In [ ]:
from reversible.spectral_norm import SpectralNorm
from torch import nn
class ConcatReLU(nn.Module):
    def __init__(self):
        super(ConcatReLU, self).__init__()

    def forward(self, x):
        return th.cat((nn.functional.relu(x), -nn.functional.relu(-x)), dim=1)

def create_adv_2_layer(dim, intermediate_dim, snorm):
    if snorm is None:
        model = nn.Sequential(
            nn.Linear(dim, intermediate_dim),
            ConcatReLU(),
            nn.Linear(intermediate_dim*2, intermediate_dim*2),
            nn.ReLU(),
            nn.Linear(intermediate_dim*2, 1))
    else:
        assert isinstance(snorm, float)
        model = nn.Sequential(
            SpectralNorm(nn.Linear(dim, intermediate_dim), power_iterations=1, to_norm=snorm),
            ConcatReLU(),
            SpectralNorm(nn.Linear(intermediate_dim*2, intermediate_dim*2), power_iterations=1, to_norm=snorm),
            nn.ReLU(),
            SpectralNorm(nn.Linear(intermediate_dim*2, 1), power_iterations=1, to_norm=snorm))

    return model



In [ ]:
from braindecode.torch_ext.util import set_random_seeds
set_random_seeds(20181102, False)
adversary = create_adv_2_layer(1,100,snorm=None)

test_inputs = np_to_var(np.linspace(-3,3,200)[:,None], dtype=np.float32)

test_outs = adversary(test_inputs)
plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_outs)[:,0])

optimizer = th.optim.Adam(adversary.parameters(), lr=1e-3)
test_inputs = np_to_var(np.linspace(-3,3,200)[:,None], dtype=np.float32)

rng = RandomState(20181102)

In [ ]:
n_epochs = 10000
for i_epoch in range(n_epochs):
    if i_epoch % (n_epochs // 10) == 0:
        fig = plt.figure(figsize=(12,2))
        test_inputs = np_to_var(np.linspace(-3,3,200)[:,None], dtype=np.float32)

        test_outs = adversary(test_inputs)
        plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_outs)[:,0])
        display(fig)
        plt.close(fig)
    fake_x = np_to_var(rng.rand(2,1) * 3-1, dtype=np.float32)
    fake_x = np_to_var(rng.rand(2,1) * 0.8+0.1, dtype=np.float32)

    out_real = adversary(x)
    out_fake = adversary(fake_x)

    score = out_real - out_fake
    loss = -(score.mean())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    

In [ ]:
# Right now: decrease value in center, increase on outside

In [ ]:
from braindecode.torch_ext.util import set_random_seeds
set_random_seeds(20181102, False)
adversary = create_adv_2_layer(1,100,snorm=None)

test_inputs = np_to_var(np.linspace(-3,3,200)[:,None], dtype=np.float32)

test_outs = adversary(test_inputs)
plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_outs)[:,0])

rng = RandomState(20181102)

optimizable_inputs = np_to_var(np.linspace(-3,3,200)[:,None], dtype=np.float32, requires_grad=True)

optimizer = th.optim.Adam([dict(params=adversary.parameters(), lr=1e-3),
                          dict(params=[optimizable_inputs], lr=-1e-3)])
plt.plot(var_to_np(optimizable_inputs)[:,0], var_to_np(optimizable_inputs)[:,0] * 0,
         ls='', marker='o', alpha=0.3, markersize=3)

In [ ]:
n_epochs = 1000
for i_epoch in range(n_epochs):
    if i_epoch % (n_epochs // 10) == 0:
        fig = plt.figure(figsize=(12,2))
        test_inputs = np_to_var(np.linspace(-3,3,200)[:,None], dtype=np.float32)

        test_outs = adversary(test_inputs)
        plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_outs)[:,0])
        plt.plot(var_to_np(optimizable_inputs)[:,0], var_to_np(optimizable_inputs)[:,0] * 0,
                 ls='', marker='o', alpha=0.3, markersize=3)
        display(fig)
        plt.close(fig)
    fake_x = np_to_var(rng.rand(2,1) * 3-1, dtype=np.float32)
    fake_x = np_to_var(rng.rand(2,1) * 0.8+0.1, dtype=np.float32)

    out_real = adversary(x)
    out_fake = adversary(optimizable_inputs)

    score = out_real.mean() - out_fake.mean()
    loss = -(score.mean())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    

In [ ]:
import torch as th
from torch import nn
from torch.nn.parameter import Parameter
from torch.nn.utils.weight_norm import _norm

class WeightNorm(object):
    def __init__(self, name, dim, fixed_norm):
        self.name = name
        self.dim = dim
        self.fixed_norm = fixed_norm

    def compute_weight(self, module):
        v = getattr(module, self.name + '_v')
        if self.fixed_norm is None:
            g = getattr(module, self.name + '_g')
        else:
            g = self.fixed_norm
        normed_weight = v * (g / _norm(v, self.dim))
        return normed_weight

    @staticmethod
    def apply(module, name, dim, fixed_norm):
        fn = WeightNorm(name, dim, fixed_norm)

        weight = getattr(module, name)

        # remove w from parameter list
        del module._parameters[name]

        # add g and v as new parameters and express w as g/||v|| * v
        if fixed_norm is None:
            module.register_parameter(name + '_g', Parameter(_norm(weight, dim).data))
        module.register_parameter(name + '_v', Parameter(weight.data))
        setattr(module, name, fn.compute_weight(module))

        # recompute weight before every forward()
        module.register_forward_pre_hook(fn)

        return fn

    def remove(self, module):
        weight = self.compute_weight(module)
        delattr(module, self.name)
        if self.fixed_norm is None:
            del module._parameters[self.name + '_g']
        del module._parameters[self.name + '_v']
        module.register_parameter(self.name, Parameter(weight.data))

    def __call__(self, module, inputs):
        setattr(module, self.name, self.compute_weight(module))


def weight_norm(module, name='weight', dim=0, fixed_norm=None):
    r"""Applies weight normalization to a parameter in the given module.

    .. math::
         \mathbf{w} = g \dfrac{\mathbf{v}}{\|\mathbf{v}\|}

    Weight normalization is a reparameterization that decouples the magnitude
    of a weight tensor from its direction. This replaces the parameter specified
    by `name` (e.g. "weight") with two parameters: one specifying the magnitude
    (e.g. "weight_g") and one specifying the direction (e.g. "weight_v").
    Weight normalization is implemented via a hook that recomputes the weight
    tensor from the magnitude and direction before every :meth:`~Module.forward`
    call.

    By default, with `dim=0`, the norm is computed independently per output
    channel/plane. To compute a norm over the entire weight tensor, use
    `dim=None`.

    See https://arxiv.org/abs/1602.07868

    Args:
        module (nn.Module): containing module
        name (str, optional): name of weight parameter
        dim (int, optional): dimension over which to compute the norm

    Returns:
        The original module with the weight norm hook

    Example::

        >>> m = weight_norm(nn.Linear(20, 40), name='weight')
        Linear (20 -> 40)
        >>> m.weight_g.size()
        torch.Size([40, 1])
        >>> m.weight_v.size()
        torch.Size([40, 20])

    """
    WeightNorm.apply(module, name, dim, fixed_norm=fixed_norm)
    return module

In [ ]:
from braindecode.torch_ext.util import set_random_seeds
set_random_seeds(20181102, False)

dim = 1
intermediate_dim = 100
norm_a = np_to_var(np.ones((intermediate_dim,1)), dtype=np.float32, requires_grad=True)
norm_b = np_to_var(np.ones((intermediate_dim * 2,1)), dtype=np.float32, requires_grad=True)
norm_c = np_to_var(np.ones((1,1)), dtype=np.float32, requires_grad=True)
adversary = nn.Sequential(
    weight_norm(nn.Linear(dim, intermediate_dim), fixed_norm=norm_a),
    ConcatReLU(),
    nn.Linear(intermediate_dim*2, intermediate_dim*2),
    nn.ReLU(),
    nn.Linear(intermediate_dim*2, 1))

test_inputs = np_to_var(np.linspace(-3,3,200)[:,None], dtype=np.float32)

test_outs = adversary(test_inputs)
plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_outs)[:,0])

rng = RandomState(20181102)

optimizable_inputs = np_to_var(np.linspace(-3,3,200)[:,None], dtype=np.float32, requires_grad=True)

optimizer = th.optim.Adam([dict(params=list(adversary.parameters()) + [norm_a] , lr=1e-3),
                          dict(params=[optimizable_inputs], lr=-1e-3)])
plt.plot(var_to_np(optimizable_inputs)[:,0], var_to_np(optimizable_inputs)[:,0] * 0,
         ls='', marker='o', alpha=0.3, markersize=3)

In [ ]:
n_epochs = 1000
for i_epoch in range(n_epochs):
    if i_epoch % (n_epochs // 10) == 0:
        fig = plt.figure(figsize=(12,2))
        test_inputs = np_to_var(np.linspace(-3,3,200)[:,None], dtype=np.float32)

        test_outs = adversary(test_inputs)
        plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_outs)[:,0])
        plt.plot(var_to_np(optimizable_inputs)[:,0], var_to_np(optimizable_inputs)[:,0] * 0,
                 ls='', marker='o', alpha=0.3, markersize=3)
        display(fig)
        plt.close(fig)
    fake_x = np_to_var(rng.rand(2,1) * 3-1, dtype=np.float32)
    fake_x = np_to_var(rng.rand(2,1) * 0.8+0.1, dtype=np.float32)

    out_real = adversary(x)
    out_fake = adversary(optimizable_inputs)

    score = out_real.mean() - out_fake.mean()
    loss = -(score.mean())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    

In [ ]:
norm_a